# load datasets

In [27]:
!pip install datasets

In [28]:
from datasets import load_dataset
import pandas as pd

comparisons = load_dataset("openai/summarize_from_feedback", name="comparisons")
axis = load_dataset("openai/summarize_from_feedback", name="axis")

comparisons_train = comparisons["train"]
comparisons_val = comparisons["validation"]
axis_val = axis["validation"]
axis_test = axis["test"]


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

# filtering datasets

In [29]:
def filter_comparisons(comparisons_ds, comparisons_filtered,summary_t2id,filtered_ids):
    for i in range(len(comparisons_ds)):
        post_id = comparisons_ds[i]['info']["id"]
        summary_text_0 = comparisons_ds[i]['summaries'][0]["text"]
        summary_text_1 = comparisons_ds[i]['summaries'][1]["text"]
        id_0, id_1 = -1, -1
        if summary_text_0 in summary_t2id:
            id_0 = summary_t2id[summary_text_0]
        if summary_text_1 in summary_t2id:
            id_1 = summary_t2id[summary_text_1]
        worker_choice = comparisons_ds[i]['choice']
        worker_id = comparisons_ds[i]['worker']
        if (id_0 != -1) and (id_1 != -1):
            filtered_ids.append(id_0)
            filtered_ids.append(id_1)
            comp_dict = {'id_0': id_0,
                         'id_1': id_1,
                         'summary_text_0': summary_text_0,
                         "summary_text_1": summary_text_1,
                         "worker_choice": worker_choice,
                         "worker_id": worker_id}
            if post_id in comparisons_filtered:
                comparisons_filtered[post_id].append(comp_dict)
            else:
                comparisons_filtered[post_id] = [comp_dict]
    return comparisons_filtered, filtered_ids

def filter_axis(axis_ds, axis_filtered, summary_id, summary_t2id, repeated_summaries):
    for i in range(len(axis_ds)):
        post_id = axis_ds[i]['info']["id"]
        summary_text = axis_ds[i]['summary']["text"]
        summary_eval = axis_ds[i]['summary']["axes"]
        worker_id = axis_ds[i]['worker']
        if summary_text in summary_t2id:
            repeated_summaries += 1
        else:
            summary_t2id[summary_text] = summary_id
            summary_id += 1
        axis_dict = {'summary_id': summary_t2id[summary_text],
                    'summary_text': summary_text,
                     "summary_eval": summary_eval,
                     "worker_id": worker_id}
        if post_id in axis_filtered:
            axis_filtered[post_id].append(axis_dict)
        else:
            axis_filtered[post_id] = [axis_dict]
    return axis_filtered, summary_id, summary_t2id, repeated_summaries


In [30]:
def update_summary_t2id(comparisons_ds, summary_t2id):
    complete_summary_t2id = summary_t2id.copy()
    # Iterate over each row in the DataFrame
    for i in range(len(comparisons_ds)):
        summary_text_0 = comparisons_ds[i]['summaries'][0]["text"]
        summary_text_1 = comparisons_ds[i]['summaries'][1]["text"]

        # Check if summary_0 is not in the summary_t2id dictionary
        if summary_text_0 not in complete_summary_t2id:
            # Generate a new id for the summary
            new_id = len(complete_summary_t2id) + 1
            # Add the summary and its id to the summary_t2id dictionary
            complete_summary_t2id[summary_text_0] = new_id

        # Check if summary_1 is not in the summary_t2id dictionary
        if summary_text_1 not in complete_summary_t2id:
            # Generate a new id for the summary
            new_id = len(complete_summary_t2id) + 1
            # Add the summary and its id to the summary_t2id dictionary
            complete_summary_t2id[summary_text_1] = new_id

    # Return the updated summary_t2id dictionary
    return complete_summary_t2id

def filter_biased_comp(comparisons_ds, comparisons_filtered_biased,summary_t2id,complete_summary_t2id,filtered_ids_biased):
    for i in range(len(comparisons_ds)):
        post_id = comparisons_ds[i]['info']["id"]
        summary_text_0 = comparisons_ds[i]['summaries'][0]["text"]
        summary_text_1 = comparisons_ds[i]['summaries'][1]["text"]
        #id_0, id_1 = -1, -1
        if summary_text_0 in summary_t2id and summary_text_1 in summary_t2id:
            id_0, id_1 = -1, -1
        else:
            id_0 = complete_summary_t2id[summary_text_0]
            id_1 = complete_summary_t2id[summary_text_1]
        worker_choice = comparisons_ds[i]['choice']
        worker_id = comparisons_ds[i]['worker']
        if (id_0 != -1) and (id_1 != -1):
            filtered_ids.append(id_0)
            filtered_ids.append(id_1)
            comp_dict = {'id_0': id_0,
                         'id_1': id_1,
                         'summary_text_0': summary_text_0,
                         "summary_text_1": summary_text_1,
                         "worker_choice": worker_choice,
                         "worker_id": worker_id}
            if post_id in comparisons_filtered_biased:
                comparisons_filtered_biased[post_id].append(comp_dict)
            else:
                comparisons_filtered_biased[post_id] = [comp_dict]
    return comparisons_filtered_biased, filtered_ids_biased


In [58]:
# Create a dictionary to map post ids to posts
post_map = {}

# Loop through the dataset and populate the dictionary
for post_data in comparisons_val["info"]:
    post_id = post_data["id"]
    post_content = post_data["post"]
    post_map[post_id] = post_content



In [31]:
# filtering axis:
axis_filtered = {}
summary_id, repeated_summaries = 0, 0
summary_t2id = {}
axis_filtered, summary_id, summary_t2id, repeated_summaries = filter_axis(axis_val, axis_filtered, summary_id, summary_t2id,repeated_summaries)
#axis_filtered, summary_id, summary_t2id, repeated_summaries = filter_axis(axis_test, axis_filtered, summary_id, summary_t2id,repeated_summaries)

# filtering comparisons: post_id, summary_text, choice_worker, worker_id, confidence
comparisons_filtered = {}
filtered_ids = []
#comparisons_filtered, filtered_ids = filter_comparisons(comparisons_train, comparisons_filtered, summary_t2id,filtered_ids)
comparisons_filtered, filtered_ids = filter_comparisons(comparisons_val, comparisons_filtered, summary_t2id,filtered_ids)
# 739 unique summaries

In [6]:
filtered_ids_biased = []
comparisons_filtered_biased = {}
complete_summary_t2id = update_summary_t2id(comparisons_val, summary_t2id)
comp_biased= filter_biased_comp(comparisons_val, comparisons_filtered_biased, summary_t2id,complete_summary_t2id,filtered_ids_biased)

In [7]:
#save results
comp = pd.concat({k: pd.DataFrame(v) for k, v in comparisons_filtered.items()})
axis_ = pd.concat({k: pd.DataFrame(v) for k, v in axis_filtered.items()})
comp_biased = pd.concat({k: pd.DataFrame(v) for k, v in comparisons_filtered_biased.items()})
axis_f = axis_[axis_['summary_id'].isin(filtered_ids)]
comp.to_csv('comp.csv')
axis_f.to_csv('eval.csv')
comp_biased.to_csv('comp_biased.csv')

# synthetic data generation

In [32]:
import pandas as pd
import ast

comp_df = pd.read_csv('comp.csv')
eval_df = pd.read_csv('eval.csv')
comp_df.rename(columns={'Unnamed: 0': 'post_id'}, inplace=True)
comp_df.rename(columns={'Unnamed: 1': 'comp_id_of_post'}, inplace=True)

eval_df.rename(columns={'Unnamed: 0': 'post_id'}, inplace=True)
eval_df.rename(columns={'Unnamed: 1': 'eval_id_of_post'}, inplace=True)
eval_df['summary_eval'] = eval_df['summary_eval'].apply(ast.literal_eval)

In [33]:
comp__unbiased_df = pd.read_csv('comp_biased.csv')
comp__unbiased_df.rename(columns={'Unnamed: 0': 'post_id'}, inplace=True)
comp__unbiased_df.rename(columns={'Unnamed: 1': 'comp_id_of_post'}, inplace=True)

In [34]:
# Extract the 'summary_eval' column as a DataFrame
summary_eval_df = pd.DataFrame(eval_df['summary_eval'].tolist())

# Drop the 'compatible' column if present
summary_eval_df = summary_eval_df.drop('compatible', axis=1)

# Concatenate the original DataFrame without 'summary_eval' column and the extracted columns
eval_table = pd.concat([eval_df.drop('summary_eval', axis=1), summary_eval_df], axis=1)

In [35]:
# Calculate the average and standard deviation for multiple evaluations
overall_stats = eval_table.groupby('summary_id')['overall'].agg(['mean', 'std'])
overall_stats = overall_stats[overall_stats['std'].notna()]  # Exclude summaries with single evaluations
accuracy_stats = eval_table.groupby('summary_id')['accuracy'].agg(['mean', 'std'])
accuracy_stats = accuracy_stats[accuracy_stats['std'].notna()]  # Exclude summaries with single evaluations
coverage_stats = eval_table.groupby('summary_id')['coverage'].agg(['mean', 'std'])
coverage_stats = coverage_stats[coverage_stats['std'].notna()]  # Exclude summaries with single evaluations
coherence_stats = eval_table.groupby('summary_id')['coherence'].agg(['mean', 'std'])
coherence_stats = coherence_stats[coherence_stats['std'].notna()]  # Exclude summaries with single evaluations


In [36]:
unique_eval = eval_table.groupby('summary_id').mean().drop("eval_id_of_post", axis=1).reset_index()
unique_eval

<ipython-input-36-67d1caedd44f>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  unique_eval = eval_table.groupby('summary_id').mean().drop("eval_id_of_post", axis=1).reset_index()


,summary_id,overall,accuracy,coverage,coherence
0,6,4.0,4.0,4.0,7.0
1,23,7.0,7.0,7.0,7.0
2,28,4.0,6.0,4.0,4.0
3,33,1.0,1.0,1.0,6.0
4,41,4.0,4.0,4.0,7.0
...,...,...,...,...,...
1343,7676,7.0,7.0,7.0,7.0
1344,7686,6.0,7.0,6.0,7.0
1345,7691,5.0,7.0,6.0,5.0
1346,7709,1.0,1.0,1.0,7.0


In [37]:
import random
import numpy as np

class Worker:
    def __init__(self, id, reliability):
        self.id = id
        self.reliability = reliability

class Summary:
    def __init__(self, id, overall, accuracy, coverage, coherence):
        self.id = id
        self.overall = overall
        self.accuracy = accuracy
        self.coverage = coverage
        self.coherence = coherence

#help functions

def assign_answer(reliability, num_comparisons):
    correct_answers = int(reliability * num_comparisons)  # Number of correct answers based on worker's reliability
    incorrect_answers = num_comparisons - correct_answers  # Number of incorrect answers
    # Create a list of correct and incorrect answers
    answers = ['correct'] * correct_answers + ['incorrect'] * incorrect_answers
    random.shuffle(answers)

    return answers

def assign_labels(worker, reliability,ground_truth_labels):

    num_labels = len(ground_truth_labels)
    #calls the assign_answer function to generate answers
    #and then creates labels by mapping the correct answers to the corresponding ground truth labels
    #and flipping the incorrect answers
    answers = assign_answer(reliability, num_labels)
    labels = [0] * num_labels
    for i in range(num_labels):
        if answers[i] == "correct":
            labels[i] = ground_truth_labels[i]
        else:
            labels[i] = 1 - ground_truth_labels[i]
    return labels

def match_worker_with_summary(num_assign_workers, worker_comp_id_list, comparison_ds):

  comp_row_indexes= random.choices(range(num_comaprisons), k=num_assign_workers)
  #match_df=comparison_ds[["id_0","id_1","worker_choice"]].iloc[comp_row_indexes]
  match_df=comparison_ds.iloc[comp_row_indexes]
  match_df['worker_id'] = worker_comp_id_list

  return match_df

def get_new_comp_labels_df(match_df,worker,reliability): #for one worker
  ground_truth_labels = match_df.loc[match_df['worker_id'] == worker.id, 'worker_choice'].tolist()
  labels=assign_labels(worker, reliability,ground_truth_labels)
  match_df['generated_label'] = None
  match_df.loc[match_df['worker_id'] == worker.id, "generated_label"] = labels
  matching_rows = match_df.loc[match_df['worker_id'] == worker.id]
  return matching_rows

def match_id_with_eval(summary_id, cirteria):
   matched_element = unique_eval.loc[unique_eval["summary_id"] == summary_id, cirteria]
   return matched_element

def randomly_assign_workers(total_num_workers, num_assign_workers, num_comparisons):
    worker_comp_id_list = []

    for worker_id in range(total_num_workers):
        if len(worker_comp_id_list) < num_assign_workers:
            remaining_comparisons = num_assign_workers - len(worker_comp_id_list)
            comparisons_per_worker = random.randint(1, int(300))  # Randomly adjust comparisons
            num_assignments = min(comparisons_per_worker, remaining_comparisons)
            worker_comp_id_list.extend([worker_id] * num_assignments)

    random.shuffle(worker_comp_id_list)  # Shuffle the list for random assignment


    return worker_comp_id_list



In [68]:
def get_the_generated_df_for_comp(total_num_workers, num_assign_workers, reliability, num_comaprisons, comparison_ds):
    worker_comp_id_list = randomly_assign_workers(total_num_workers, num_assign_workers ,num_comaprisons)
    match_df = match_worker_with_summary(num_assign_workers, worker_comp_id_list, comparison_ds)
    generated_df = pd.DataFrame()
    matching_rows = pd.DataFrame()

    for worker_id in set(worker_comp_id_list):
        if reliability == "low":
            worker = Worker(worker_id, 0.2)  # Set low reliability for all workers
            matching_rows = get_new_comp_labels_df(match_df, worker, 0.2)
        elif reliability == "medium":
            worker = Worker(worker_id, 0.5)  # Set 50% reliability for all workers
            matching_rows = get_new_comp_labels_df(match_df, worker, 0.5)
        elif reliability == "high":
            worker = Worker(worker_id, 0.8)  # Set 80% reliability for all workers
            matching_rows = get_new_comp_labels_df(match_df, worker, 0.8)
        elif reliability == "half_low_half_high":
            # Assign 20% reliability to half of the workers, and 80% reliability to the other half
            if worker_id < total_num_workers // 2:
                worker = Worker(worker_id, 0.2)
                matching_rows = get_new_comp_labels_df(match_df, worker, 0.2)
            else:
                worker = Worker(worker_id, 0.8)
                matching_rows = get_new_comp_labels_df(match_df, worker, 0.8)
        elif reliability == "quarter_reliabilities":
            # Assign different reliabilities to each 25% of workers
            num_quarters = 4
            reliability_options = [0.2, 0.4, 0.6, 0.8]
            quarter_size = total_num_workers / num_quarters
            quarter_index = int(worker_id // quarter_size)
            worker = Worker(worker_id, reliability_options[quarter_index])
            matching_rows = get_new_comp_labels_df(match_df, worker, reliability_options[quarter_index])
        elif reliability == "perfect":
            worker = Worker(worker_id, 1.0)  # Set perfect reliability for all workers
            matching_rows = get_new_comp_labels_df(match_df, worker, 1.0)
        elif reliability == "random":
            # Handle random scenarios here
            rj = random.uniform(0.0, 1.0)
            worker = Worker(worker_id, rj)
            matching_rows = get_new_comp_labels_df(match_df, worker, rj)

        generated_df = pd.concat([generated_df, matching_rows])


    return generated_df

In [39]:
def introduce_bias(comp_df_input, bias_accuracy, bias_coverage, bias_coherence, bias_towards_0, bias_towards_1):
    if bias_accuracy:
        for i in range(len(comp_df_input)):
            accuracy_0 = float(match_id_with_eval(comp_df_input.loc[i, "id_0"], "accuracy"))
            accuracy_1 = float(match_id_with_eval(comp_df_input.loc[i, "id_1"], "accuracy"))
            if accuracy_0 > accuracy_1:
                comp_df_input.loc[i, "worker_label"] = 0
            elif accuracy_0 < accuracy_1:
                comp_df_input.loc[i, "worker_label"] = 1

    if bias_coverage:
        for i in range(len(comp_df_input)):
            coverage_0 = float(match_id_with_eval(comp_df_input.loc[i, "id_0"], "coverage"))
            coverage_1 = float(match_id_with_eval(comp_df_input.loc[i, "id_1"], "coverage"))
            if coverage_0 > coverage_1:
                comp_df_input.loc[i, "generated_label"] = 0
            elif coverage_0 < coverage_1:
                comp_df_input.loc[i, "worker_label"] = 1

    if bias_coherence:
        for i in range(len(comp_df_input)):
            coherence_0 = float(match_id_with_eval(comp_df_input.loc[i, "id_0"], "coherence"))
            coherence_1 = float(match_id_with_eval(comp_df_input.loc[i, "id_1"], "coherence"))
            if coherence_0 > coherence_1:
                comp_df_input.loc[i, "worker_label"] = 0
            elif coherence_0 < coherence_1:
                comp_df_input.loc[i, "worker_label"] = 1

    if bias_towards_0:
        for i in range(len(comp_df_input)):
           comp_df_input.loc[i, "worker_label"] = 0

    if bias_towards_1:
        for i in range(len(comp_df_input)):
           comp_df_input.loc[i, "worker_label"] = 1



    return comp_df_input


In [60]:
def to_json(generated_df, reliability):
  df_store = pd.DataFrame()

  for index, row in generated_df.iterrows():

    new_row_data = {
      "info": {
          "id": row["post_id"],
          "post": post_map[row["post_id"]],
          "title": None,
          "subreddit": None
      },
      "split": "train",
      "summaries": [
          {
              "text": row["summary_text_0"],
              "policy": None,
              "note": None,
          },
          {
              "text": row["summary_text_1"],
              "policy": None,
              "note": None,
          }
      ],
      "choice": row["worker_label"],
      "worker": row["worker_id"],
      "batch": "batch1",
      "extra": {
          "expert_label": row["expert_label"],
          "reliability": reliability

      }
    }

    df_store = df_store.append(new_row_data, ignore_index=True)
    #df_store = pd.concat([df_store, pd.DataFrame(new_row_data, index=[0])], ignore_index=True)
  file_path = reliability + ".json"
  df_store.to_json(file_path, orient="records", lines=True)






In [63]:
def to_json_biased(generated_df, reliability, bias_type):
  df_store = pd.DataFrame()

  for index, row in generated_df.iterrows():

    new_row_data = {
      "info": {
          "id": row["post_id"],
          "post": post_map[row["post_id"]],
          "title": None,
          "subreddit": None
      },
      "split": "train",
      "summaries": [
          {
              "text": row["summary_text_0"],
              "policy": None,
              "note": None,
          },
          {
              "text": row["summary_text_1"],
              "policy": None,
              "note": None,
          }
      ],
      "choice": row["worker_label"],
      "worker": row["worker_id"],
      "batch": "batch1",
      "extra": {
          "expert_label": row["expert_label"],
          "reliability": reliability,
          "bias": bias_type
      }
    }

    df_store = df_store.append(new_row_data, ignore_index=True)
    #df_store = pd.concat([df_store, pd.DataFrame(new_row_data, index=[0])], ignore_index=True)
  file_path = bias_type + ".json"
  df_store.to_json(file_path, orient="records", lines=True)
  return df_store


In [61]:
#unbiased
num_comaprisons=6543  #727/0.3*0.7
total_num_experts=56   #comp__biased_df[:6543]['worker_id'].nunique()
total_num_workers=56 * 3
num_assign_workers=6543*3 #total number of comparions we want to generate
reliability="random"
generated_df = get_the_generated_df_for_comp(total_num_workers,num_assign_workers, reliability,num_comaprisons, comp__unbiased_df[:6543]).reset_index()
generated_df.rename(columns={'generated_label': 'worker_label','worker_choice': 'expert_label'}, inplace=True)

<ipython-input-37-0d26127aedfd>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df['worker_id'] = worker_comp_id_list
<ipython-input-37-0d26127aedfd>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df['generated_label'] = None
<ipython-input-37-0d26127aedfd>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

In [62]:
#unbiased
a= to_json(generated_df, reliability)

Streaming output truncated to the last 5000 lines.
<ipython-input-60-d8126a6cad33>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_store = df_store.append(new_row_data, ignore_index=True)
<ipython-input-60-d8126a6cad33>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_store = df_store.append(new_row_data, ignore_index=True)
<ipython-input-60-d8126a6cad33>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_store = df_store.append(new_row_data, ignore_index=True)
<ipython-input-60-d8126a6cad33>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_store = df_store.append(new_row_data, ignore_index=True)
<ipython-input-60-d8126a6

In [77]:
#biased
num_comaprisons=727
total_num_experts=59
total_num_workers=59 * 3
num_assign_workers=727*3 #total number of comparions we want to generate
reliability="perfect"
generated_df = get_the_generated_df_for_comp(total_num_workers,num_assign_workers, reliability,num_comaprisons, comp_df).reset_index()
generated_df.rename(columns={'generated_label': 'worker_label','worker_choice': 'expert_label'}, inplace=True)

#assign bias
bias_accuracy = False  # Bias towards highly accurate summaries
bias_coverage = False # Bias towards high coverage summaries
bias_coherence = False  # Bias towards highly coherent summaries
bias_towards_0 = False   #Bias towards class 0 over class 1
bias_towards_1 = True   #Bias towards class 1 over class 0
generated_df = introduce_bias(generated_df, bias_accuracy, bias_coverage, bias_coherence, bias_towards_0, bias_towards_1)
generated_df

<ipython-input-37-0d26127aedfd>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df['worker_id'] = worker_comp_id_list
<ipython-input-37-0d26127aedfd>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_df['generated_label'] = None
<ipython-input-37-0d26127aedfd>:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

,index,post_id,comp_id_of_post,id_0,id_1,summary_text_0,summary_text_1,expert_label,worker_id,worker_label
0,144,t3_1zcu45,15,4112,4109,My girlfriend doesn't know what she wants aft...,My girlfriend wants to break up because she t...,0,0,1
1,551,t3_yqkdq,2,4964,1485,"I'm a doormat when it comes to anger, my boyf...",Agression scares me. Boyfriend is on the agre...,0,0,1
2,139,t3_1zcu45,10,4109,4108,My girlfriend wants to break up because she t...,My girlfriend doesn't know what she wants aft...,0,0,1
3,537,t3_4p8xqe,0,4752,4748,Me [24 /F] interested on my surf instructor [...,what do you guys think? I mean if you were a ...,0,0,1
4,481,t3_330ndi,0,6464,6462,I think my ex-boyfriend friend likes me. He s...,Was hanging out with ex-boyfriend. He seemed ...,0,0,1
...,...,...,...,...,...,...,...,...,...,...
2176,254,t3_4btpvt,4,5662,5661,I want to date my female friend with benefits...,What should I do? Should I Cut her off and lo...,0,17,1
2177,414,t3_49nwvy,0,2646,2649,employer is making changes to my contract and...,Employer wants to amend my contract to includ...,0,17,1
2178,412,t3_45p2ab,1,6136,6138,Husband lied to me about going on a trip with...,Husband lied to me about going on a trip with...,0,17,1
2179,666,t3_123h65,0,923,919,A major university sent me an acceptance pack...,Got into a major university that I didn't app...,1,17,1


In [ ]:
bia= to_json_biased(generated_df, reliability, "bias_towards_1")

In [49]:
df_original = pd.DataFrame([
    {
        "info": {
            "id": "t3_1a3ghj",
            "post": "Hello there.\nThis girl [f21] and I [f22] have been friends for almost 6 years...",
            "title": "My best friend [f21] demands too much attention and I [f22] don\u00b4t know what to do.",
            "subreddit": "relationships"
        },
        "split": "valid2",
        "summaries": [
            {
                "text": " buddy stubbornly demands constant attention despite tried compromise approach???...",
                "policy": "sup4_ppo_rm3_kl260",
                "note": "Vague and rambling. \n------------------\n- captures OP's frustrations but few details of the problem with friend - emotion more than point.\n- no financial repercussions exist\n- mentions failed compromise - key\n- captures purpose well\n- extremely poor conveyance of OP's voice (style)"
            },
            {
                "text": " My best friend demands too much attention and knows how to manipulate me...",
                "policy": "sup4_ppo_rm3_kl6",
                "note": "Clearly a backstory to the issues that does not come through...",
            }
        ],
        "choice": 0,
        "worker": "OKFDIsAZl6Qa0m9x26f5Ao4S0uc7Ca",
        "batch": "batch11",
        "extra": {
            "confidence": 4
        }
    },
    {
        "info": {
            "id": "t3_1a3ghj",
            "post": "Hello there.\nThis girl [f21] and I [f22] have been friends for almost 6 years...",
            "title": "My best friend [f21] demands too much attention and I [f22] don\u00b4t know what to do.",
            "subreddit": "relationships"
        },
        "split": "valid2",
        "summaries": [
            {
                "text": " Best friend [f21] needs a lot of attention and I cannot give it to her, I don't want to hurt her but I don't know what to do.",
                "policy": "ref",
                "note": "Unclear on what grounds OP fears hurting friend. Presumably by not giving required attention. Leaves OP stuck between two negative options.\n-----------\n- 'don't want to hurt her' is off, OP wants to stop feeling so depended upon when she is unable to give the support the friend needs.\n- does not capture the compromise "
            },
            {
                "text": " My best friend demands too much attention and I don\u00b4t know how to stop feeling like I am walking among eggshells with her.",
                "policy": "sup4_ppo_rm3_kl2",
                "note": "Unclear why delicacy is required of OP - suggests something very negative occurs if OP is not delicate in their dealings with friend.\n-------------\n- no mention of failed compromise\n- Purpose poorly conveyed."
            }
        ],
        "choice": 1,
        "worker": "OKFDIsAZl6Qa0m9x26f5Ao4S0uc7Ca",
        "batch": "batch11",
        "extra": {
            "confidence": 8
        }
    }
])